In [6]:
using Pkg
Pkg.activate(".") # Activate the environment
using ArgParse
using Logging, LoggingExtras
using JSON
using LegendHDF5IO
using FilePathsBase
using DataStructures
using PropDicts
using Tables
using TypedTables
using LegendDataManagement
using Printf

  Activating project at `~/LEGEND/ZeroNuFit.jl`


### Notebook to make the configs for the fit
This should be converted to a `script.jl` all it does it reformat inputs from L-200, GERDA and Majorana into one common format readble by `ZeroNuFit.jl`

### Events files
First we need files containing the physics events in the three experiments.
For L-200 these are not available to the collaboration, so we need to create the file.

In [17]:
function read_events_gerda(file_path)
    timestamp=[1455109448, 1457847659, 1472522222, 1475981084, 1480290460,
                   1485848926, 1503578885, 1509498133, 1516142805, 1533092526,
                   1539047354, 1566823934, 1568276649]
    detector=["ANG4", "GD61C", "GD35B", "ANG1", "GD35B", "GD91A", "GD76C",
                  "ANG1", "RG1", "GD61C", "IC74A", "ANG4", "GD32D"]
    energy=[1995.2452, 1958.6807, 2018.1346, 1950.9419, 2067.9735,
                2056.4280, 2042.0641, 1962.7372, 1957.5059, 1970.1398,
                2058.8776, 2015.8751, 2012.0643]
    experiment = fill("GERDA",length(energy))
    output=Dict("events" => [])
    for (exp,t,d,e) in zip(experiment,timestamp,detector,energy)
        append!(output["events"],[Dict("experiment"=>exp,"timestamp"=>t,"energy"=>e,"detector"=>d)])
    end
    open(file_path, "w") do file
        write(file, json(output,4))
    end
end

read_events_gerda("config/events_gerda.json")

Dict{String, Vector{Any}}("events" => [Dict{String, Any}("experiment" => "GERDA", "energy" => 1995.2452, "timestamp" => 1455109448, "detector" => "ANG4"), Dict{String, Any}("experiment" => "GERDA", "energy" => 1958.6807, "timestamp" => 1457847659, "detector" => "GD61C"), Dict{String, Any}("experiment" => "GERDA", "energy" => 2018.1346, "timestamp" => 1472522222, "detector" => "GD35B"), Dict{String, Any}("experiment" => "GERDA", "energy" => 1950.9419, "timestamp" => 1475981084, "detector" => "ANG1"), Dict{String, Any}("experiment" => "GERDA", "energy" => 2067.9735, "timestamp" => 1480290460, "detector" => "GD35B"), Dict{String, Any}("experiment" => "GERDA", "energy" => 2056.428, "timestamp" => 1485848926, "detector" => "GD91A"), Dict{String, Any}("experiment" => "GERDA", "energy" => 2042.0641, "timestamp" => 1503578885, "detector" => "GD76C"), Dict{String, Any}("experiment" => "GERDA", "energy" => 1962.7372, "timestamp" => 1509498133, "detector" => "ANG1"), Dict{String, Any}("experiment

2072

In [24]:
function read_events_l200(file_path,meta_path)
    files =readdir("data/l200/skm/phy/")
   
    files=[joinpath("data/l200/skm/phy/",f) for f in files]
    
    vsel=ValiditySelection("20230312T043356Z", :phy)
    chmap = LegendDataManagement.AnyProps(meta_path).hardware.configuration.channelmaps(vsel)
    map = Dict( chmap[name].daq.rawid => String(name) for name in keys(chmap))
    E_fit=[]
    t_fit=[]
    d_fit=[]
    for f in files
        data = lh5open(f)["skm"][:]
        
        data   =data[.!data.coincident.muon_offline .&&
            .!data.coincident.spms .&&
            data.geds.psd.is_bb_like
        ]
    
        E = data.geds.energy
        T=  data.trigger.timestamp
        R= data.geds.rawid
        # cannot code a union (∪) of disjoint intervals with IntervalSets?
        ranges =[[1930,2099],[2109,2114],[2124,2190]]
       
        for (e,t,ra) in zip(E,T,R)
            for r in ranges
                if (e<r[2] && e>r[1])
                    append!(E_fit,e)
                    append!(t_fit,t)
                    append!(d_fit,[map[Int(ra)]])
                end
               
             end
        end
  
    end
    blind_energy = [2040.261963,2016.760010]
    blind_ts = [1693877463,1698552984]
    blind_det =["P00661C","V00048A"]
    append!(E_fit,blind_energy)
    append!(t_fit,blind_ts)
    append!(d_fit,blind_det)

    output=Dict("events" => [])
    for (t,d,e) in zip(t_fit,d_fit,E_fit)
        append!(output["events"],[Dict("experiment"=>"L200","timestamp"=>t,"energy"=>e,"detector"=>d)])
    end
    open(file_path, "w") do file
        write(file, json(output,4))
    end
end
read_events_l200("config/events_l200.json","../legend-metadata/")

1186

In [30]:
function read_events_mjd(file_path,output_path)


    E_fit=[]
    t_fit=[]
    d_fit=[]
    ranges =[[1930,2099],[2109,2114],[2124,2190]]

    open(file_path, "r") do file
    
        for line in eachline(file)
            ls=split(line," ")
            name = ls[1]
            ts=parse(Int,ls[2])
            energy =parse(Float64,ls[3])
            bkg = ls[4]
            for r in ranges
                if (energy<r[2] && energy>r[1])
                    append!(E_fit,energy)
                    append!(t_fit,ts)
                    append!(d_fit,[name])
                end
            end
        end
    end
    
   
    output=Dict("events" => [])
    for (t,d,e) in zip(t_fit,d_fit,E_fit)
        append!(output["events"],[Dict("experiment"=>"MJD","timestamp"=>t,"energy"=>e,"detector"=>d)])
    end
    open(output_path, "w") do file
        write(file, json(output,4))
    end
end
read_events_mjd("data/mjd/events.txt","config/events_mjd_new_part.json")

14485

In [22]:
str="ABC"
str[1]*str[2]

"AB"

In [23]:
function gerda_name_to_type(name)
    if (name[1]*name[2]=="GD")
        return "BG"
    elseif (name[1]*name[2]=="RG" || name[1]*name[2]=="AN")
        return "SC"
    elseif (name[1]*name[2]=="IC")
        return "IC"
    end
end

gerda_name_to_type (generic function with 1 method)

In [27]:
gerda_name_to_type("IC11")

"IC"

### Partitions files

In [37]:
function make_partitions_file_gerda(input_path,output_path,by_type=false)
    json_data = JSON.parsefile(input_path,dicttype=DataStructures.OrderedDict)

    if (by_type==false)
        fit_groups=Dict("all_phase_II"=>Dict("range"=>[[1930,2099],[2109,2114],[2124,2190]],"model"=>"uniform",
            "bkg_name"=>"B_gerda_all_pII"))
    else
        fit_groups=Dict()
        for dettype in ["BG","IC","SC"]
            fit_groups["$(dettype)_phase_II"]=Dict("range"=>[[1930,2099],[2109,2114],[2124,2190]],"model"=>"uniform",
            "bkg_name"=>"B_gerda_$(dettype)_pII")
        end
    end
    partitions_full=Dict()
    for group in keys(fit_groups)
        partitions_full[group]=[]
        counter=0
        for (part,part_data) in sort(json_data)
            part_name = String("part")*@sprintf("%02d", counter)
            counter+=1
            println(part_name)
            for (det,info) in json_data[part]
                if (info isa OrderedDict)
                    if (by_type && gerda_name_to_type(det)!=group[1]*group[2])
                        continue
                    end
                    part_temp = OrderedDict("experiment"=>"GERDA","detector"=> det,
                                    "part_name"=>part_name,
                                     "start_ts"=>json_data[part]["start_ts"],
                                     "end_ts"=>json_data[part]["end_ts"])
                    copy_fields=["eff_tot","eff_tot_sigma","fwhm","fwhm_sigma","exposure","bias"]
                    for field in copy_fields
                        part_temp[field]=json_data[part][det][field]
                    end
                       
                    part_temp["bias_sigma"]=json_data[part][det]["energy_sigma"]
                    append!(partitions_full[group],[part_temp])
                end
            end
        end
        
    end
    output=Dict("fit_groups"=>fit_groups,
                "partitions"=>partitions_full)
    
    open(output_path, "w") do file
        write(file, json(output,4))
      
    end
                
end

make_partitions_file_gerda (generic function with 2 methods)

In [38]:
make_partitions_file_gerda("/home/tdixon/Downloads/0vbb-analysis-parameters.json","config/partitions_gerda_by_type.json",true)

part00
part01
part02
part03
part04
part05
part06
part07
part08
part09
part10
part00
part01
part02
part03
part04
part05
part06
part07
part08
part09
part10
part00
part01
part02
part03
part04
part05
part06
part07
part08
part09
part10


201423

In [7]:
function printdb(db)
        if db isa PropDict
            for (key,value) in db
                println(key)
                printdb(value)
            end
        else
            println("\t",db)
            println()
        end
    end

printdb (generic function with 1 method)

In [30]:
Sys.BINDIR
Pkg.envdir()
Base.active_project()
LOAD_PATH

3-element Vector{String}:
 "@"
 "@v#.#"
 "@stdlib"

In [63]:
function make_partitions_file_l200(meta_path,output_path)

    meta = LegendDataManagement.AnyProps(meta_path)

    partitions = meta.datasets.ovbb_partitions_pars
    printflag=false
    output=[]
    # loop over detectors
    for (detector, detdata) in partitions
        if detector == :default
            continue
        end

      
        
        # apply defaults
        if partitions.default isa PropDicts.MissingProperty 
            detdata_merge = copy(detdata)
        else
            new = partitions.default
            detdata_merge = merge(new,copy(detdata))
        end
        
        # loop over partitions for this detector
        for (partition, pardata) in detdata_merge
            if partition == :default
                continue
            end

            if detdata_merge.default isa PropDicts.MissingProperty 
                pardata_merge = copy(pardata)
            else
                new = detdata_merge.default
                pardata_merge = merge(new,copy(pardata))
            end

            if printflag == true
                println("for partition $partition ")
                #printdb(pardata_merge)
                println("\n")
            end
            part_temp = OrderedDict("experiment"=>"L200","detector"=> detector,"part_name"=>partition,
                                 "start_ts"=>pardata_merge[:span_in_utc_s][1],
                                 "end_ts"=>pardata_merge[:span_in_utc_s][2])
            eff = prod([v[:val] for v in values(pardata_merge[:ovbb_acceptance])])
            eff_sigma = √(sum([v[:unc] for v in values(pardata_merge[:ovbb_acceptance])].^2))    

            # include the enrichment
            eff*=           meta.hardware.detectors.germanium.diodes[detector].production.enrichment.val
            enrichment_err =meta.hardware.detectors.germanium.diodes[detector].production.enrichment.unc

            eff_sigma=√(eff_sigma^2+enrichment_err^2)

            part_temp["eff_tot"]=eff
            part_temp["eff_tot_sigma"]=eff_sigma
            part_temp["fwhm"]=pardata_merge[:fwhm_in_keV][:val]
            part_temp["fwhm_sigma"]=pardata_merge[:fwhm_in_keV][:unc]
            part_temp["exposure"]=pardata_merge[:livetime_in_s]*
                meta.hardware.detectors.germanium.diodes[detector].production.mass_in_g/(60*60*24*365.25*1000)
            part_temp["bias"]=pardata_merge[:energy_bias_in_keV][:val]
            part_temp["bias_sigma"]=pardata_merge[:energy_bias_in_keV][:unc]

            append!(output,[part_temp])

        end
    end
    output_full=Dict("fit_groups"=>Dict("l200a"=>Dict("range"=>[[1930,2099],[2109,2114],[2124,2190]],"model"=>"uniform")),
                "partitions"=>Dict("l200a"=>output))
    open(output_path, "w") do file
        write(file, json(output_full,4))
       
    end
end


make_partitions_file_l200 (generic function with 1 method)

In [64]:
make_partitions_file_l200("../legend-metadata","config/partitions_l200.json")

154015

In [17]:
function make_partitions_mjd(input_path,output_path)

    
    
    fit_groups=["mjd-DS0","mjd-mod1","mjd-mod2"]
    partitions=OrderedDict()
    for fit in fit_groups
        partitions[fit]=[]
    end
    keywords =["detnum","detector","fwhm","fwhm_sigma","eff_tot","eff_tot_sigma","exposure","bias","bias_sigma","bkgname","sysname"]
    
    open(input_path, "r") do file
        time_start=0
        time_end=0
        for line in eachline(file)
            if startswith(line,"t")
                ts = split(line," ")
                time_start = parse(Int,ts[2])
                time_end = parse(Int,ts[3])
                continue
            else
                info = split(line," ")
                info_dict=Dict()
                for (id,key) in enumerate(keywords)
                    info_dict[key]=info[id]
                end
            end
            fit_group=info_dict["bkgname"]
            if (time_start==1000)
                part_name="first-ds"
            else
                part_name="other-ds"
            end
            part_temp=OrderedDict("experiment"=>"MJD","detector"=>info_dict["detector"],
                "part_name"=>part_name,
                "start_ts"=>time_start,"end_ts"=>time_end,
                "eff_tot"=>parse(Float64,info_dict["eff_tot"]), "eff_tot_sigma"=>parse(Float64,info_dict["eff_tot_sigma"]),
                "fwhm"=>parse(Float64,info_dict["fwhm"]), "fwhm_sigma"=>parse(Float64,info_dict["fwhm_sigma"]),
                "exposure"=>parse(Float64,info_dict["exposure"]),
                 "bias"=>parse(Float64,info_dict["bias"]), "bias_sigma"=>parse(Float64,info_dict["bias_sigma"]))

            append!(partitions[fit_group],[part_temp])


            
        end
    
        fit_groups_dict=Dict()
        for fit in fit_groups
            fit_groups_dict[fit]=Dict("bkg_name"=>fit,"range"=>[[1930,2099],[2109,2114],[2124,2190]],"model"=>"uniform")
        end

        output=Dict("fit_groups"=>fit_groups_dict,"partitions"=>partitions)
        open(output_path, "w") do file
            write(file, json(output,4))
        end
    end    
end

make_partitions_mjd (generic function with 1 method)

In [18]:
make_partitions_mjd("data/mjd/parameter.txt","config/partitions_mjd_new.json")

3150